# Calcite New Frontend Guide

This notebook will demonstrate how you can build frontends for languages other than SQL for Apache Calcite.

There are many languages for querying data. SQL is the most common, but other languages include:
- Datalog
- RDF/Graph Queries
- GraphQL
- Custom DSL's

Calcite currently does not have any tutorial materials on how to integrate alternative frontends.

This guide demonstrates how to do so, via a guided walkthrough of writing a `GraphQL -> Calcite RelNode` converter.

(If you are familiar with the Calcite `Sql2RelConverter` class -- what we will be writing is essentially a `Gql2RelConverter`)

# Overview and Goal

The aim of this notebook is to walk through the translation of a GraphQL query like this:

```graphql
query {
    EMP(
        limit: 2,
        offset: 1,
        where: {
            _or: [
                { DEPTNO: { _eq: 20 } },
                { DEPTNO: { _eq: 30 } }
            ]
            _and: [
                { SAL: { _gte: 1500 } }
                {
                    _or: [
                        { JOB: { _eq: "SALESMAN" } },
                        { JOB: { _eq: "MANAGER" } }
                    ]
                }
            ]
        }
    ) {
        EMPNO
        ENAME
        JOB
        MGR
        SAL
        COMM
        DEPTNO
    }
}
```

Into a SQL query like:

```sql
SELECT "EMPNO", "ENAME", "JOB", "MGR", "SAL", "COMM", "DEPTNO"
FROM "EMP"
WHERE "DEPTNO" IN (20, 30) AND "SAL" >= 1500 AND "JOB" IN ('MANAGER', 'SALESMAN')
```

And finally, execute that SQL statement & ensure the proper results are returned.

You will learn about the following Calcite classes/concepts:
- SqlParser
- RexNode
- RelNode
- RelToSqlConverter
- RelBuilder
- tools.Frameworks
- CalciteConnection
- Schema/SchemaPlus

# Technical Overview, 10,000ft View

What we need to do in order to accomplish our goal is roughly:

1. Parse a GraphQL **query string** into a GraphQL **Document AST**
2. Convert the GraphQL **Document AST** into some representation of **Relational Query semantics**
3. Generate a Calcite **"Row Expression" (RexNode)** from our representation of the **`WHERE` clause**
4. Create a Calcite **"Relational Expression" (RelNode)** from a combination of the **`WHERE` "RexNode" and `SELECT` "RexNode"**
5. **Execute** the RelNode against the **data source** we are trying to query against

As a picture, it looks something like this:

![GQL query translation pipeline](https://i.imgur.com/yfIVAga.png)

# Dependencies & Utility Functions (Skip this)

The following code cells simply install required libraries and define a few helper/utility functions.
You can skip this.

In [1]:
@file:DependsOn("org.apache.calcite:calcite-core:1.29.0")
@file:DependsOn("org.apache.calcite:calcite-testkit:1.29.0")
@file:DependsOn("com.graphql-java:graphql-java:17.3")
@file:DependsOn("org.hsqldb:hsqldb:2.4.1")

In [2]:
fun Any.prettyPrint(): String {
    var indentLevel = 0
    val indentWidth = 4

    fun padding() = "".padStart(indentLevel * indentWidth)
    val toString = toString()
    val stringBuilder = StringBuilder(toString.length)

    var i = 0
    while (i < toString.length) {
        when (val char = toString[i]) {
            '(', '[', '{' -> {
                indentLevel++
                stringBuilder.appendLine(char).append(padding())
            }
            ')', ']', '}' -> {
                indentLevel--
                stringBuilder.appendLine().append(padding()).append(char)
            }
            ',' -> {
                stringBuilder.appendLine(char).append(padding())
                // ignore space after comma as we have added a newline
                val nextChar = toString.getOrElse(i + 1) { char }
                if (nextChar == ' ') i++
            }
            else -> {
                stringBuilder.append(char)
            }
        }
        i++
    }

    return stringBuilder.toString()
}

# 1. Parse a GraphQL query string into a GraphQL Document AST

Below is code that we're not too interested in from the Calcite side of things.

This is necessary for the rest of the tutorial, but here we just set the groundwork for parsing a GraphQL query from a `String` into it's AST form (called a "`Document`").

In [3]:
import graphql.language.Argument
import graphql.language.OperationDefinition
import graphql.language.Field
import graphql.parser.Parser

val sampleQuery =
    """
    query {
        EMP(
            limit: 2,
            offset: 1,
            where: {
                _or: [
                    { DEPTNO: { _eq: 20 } },
                    { DEPTNO: { _eq: 30 } }
                ]
                _and: [
                    { SAL: { _gte: 1500 } }
                    {
                        _or: [
                            { JOB: { _eq: "SALESMAN" } },
                            { JOB: { _eq: "MANAGER" } }
                        ]
                    }
                ]
            }
        ) {
            EMPNO
            ENAME
            JOB
            MGR
            SAL
            COMM
            DEPTNO
        }
    }
    """

fun getFirstQuery(queryString: String): Field? {
    val document = Parser.parse(queryString)
    val root = document.definitions.first() as OperationDefinition
    val queries = root.selectionSet.selections
    return queries.first() as Field
}

fun getWhereArgumentFromGraphQLQuery(queryString: String): Argument? {
    val firstQuery = getFirstQuery(queryString)
    if (firstQuery != null) {
        return firstQuery.arguments.firstOrNull { it.name == "where" }
    }
    return null
}

fun getQuerySelectedFieldNames(queryString: String): List<String> {
    val firstQuery = getFirstQuery(queryString)
    if (firstQuery != null) {
        return firstQuery.selectionSet.selections.map { (it as Field).name }
    }
    return emptyList()
}

# 2. Convert the GraphQL Document AST into some representation of Relational Query semantics

Here, we will write some code that converts the unwieldy GQL Query Document AST into a representation that is easier for us to work with and closer to the relational expression semantics we are trying to express.

Our Expression kind hierarchy looks like this:

```scala
type Expression =
    | BinaryExpression
    | UnaryExpression
    | Column (name)
    | Literal (value)

type BinaryExpression (left, right) =
    | AND
    | OR
    | NOT
    | EQ
    | NE
    | LT
    | LTE
    | GT
    | GTE
    | IN
    | NIN

type UnaryExpression (operand) =
    | NOT
    | IS_NULL
```

Once we have a representation in this form, we can traverse it and convert it to a Calcite Row Expression (RexNode) that has identical semantics.

A useful tip here is that the signature for Calcite's `RelBuilder.call()` is:

```java
public RexNode call(SqlOperator operator, RexNode... operands) {}
public RexNode call(SqlOperator operator, Iterable<? extends RexNode> operands) {}
```

We can model our own Relational Expression types as entities that each have an attached `SqlOperator`.

For binary operations, Calcite has a type called `SqlBinaryOperator`.

Calcite doesn't have something that maps directly to "Unary" operations. It has `SqlPrefixOperation` and `SqlPostfixOperation` instead.

You can find the operators under **`org.apache.calcite.sql.fun.SqlStdOperatorTable`**.

In [4]:
import org.apache.calcite.rex.RexNode
import org.apache.calcite.sql.SqlBinaryOperator
import org.apache.calcite.sql.SqlOperator
import org.apache.calcite.sql.`fun`.SqlStdOperatorTable

sealed interface Expression {}

sealed interface BinaryOperation : Expression {
    val left: Expression
    val right: Expression
    val operation: SqlBinaryOperator
}

sealed interface UnaryOperation : Expression {
    val operand: Expression
    val operation: SqlOperator
}

data class AND(override val left: Expression, override val right: Expression) : BinaryOperation {
    override val operation: SqlBinaryOperator = SqlStdOperatorTable.AND
}

data class OR(override val left: Expression, override val right: Expression) : BinaryOperation {
    override val operation: SqlBinaryOperator = SqlStdOperatorTable.OR
}

data class EQ(override val left: Expression, override val right: Expression) : BinaryOperation {
    override val operation: SqlBinaryOperator = SqlStdOperatorTable.EQUALS
}

data class NEQ(override val left: Expression, override val right: Expression) : BinaryOperation {
    override val operation: SqlBinaryOperator = SqlStdOperatorTable.NOT_EQUALS
}

data class LT(override val left: Expression, override val right: Expression) : BinaryOperation {
    override val operation: SqlBinaryOperator = SqlStdOperatorTable.LESS_THAN
}

data class LTE(override val left: Expression, override val right: Expression) : BinaryOperation {
    override val operation: SqlBinaryOperator = SqlStdOperatorTable.LESS_THAN_OR_EQUAL
}

data class GT(override val left: Expression, override val right: Expression) : BinaryOperation {
    override val operation: SqlBinaryOperator = SqlStdOperatorTable.GREATER_THAN
}

data class GTE(override val left: Expression, override val right: Expression) : BinaryOperation {
    override val operation: SqlBinaryOperator = SqlStdOperatorTable.GREATER_THAN_OR_EQUAL
}

data class IN(override val left: Expression, override val right: Expression) : BinaryOperation {
    override val operation: SqlBinaryOperator = SqlStdOperatorTable.IN
}

data class NIN(override val left: Expression, override val right: Expression) : BinaryOperation {
    override val operation: SqlBinaryOperator = SqlStdOperatorTable.NOT_IN
}

data class NOT(override val operand: Expression) : UnaryOperation {
    override val operation: SqlOperator = SqlStdOperatorTable.NOT
}

data class IS_NULL(override val operand: Expression) : UnaryOperation {
    override val operation: SqlOperator = SqlStdOperatorTable.IS_NULL
}

@JvmInline
value class COLUMN(val name: String) : Expression

@JvmInline
value class LITERAL(val value: Any) : Expression

In [5]:
import graphql.language.*
import graphql.parser.Parser

// Unwrap a GraphQL "Value" node to it's Java representation
fun graphqlValueToJava(value: Value<*>): Any {
    return when (value) {
        is IntValue -> value.value
        is FloatValue -> value.value
        is StringValue -> value.value
        is BooleanValue -> value.isValue
        is EnumValue -> value.name
        is ArrayValue -> value.values.map { graphqlValueToJava(it) }
        else -> throw IllegalArgumentException("Unsupported value type: ${value.javaClass}")
    }
}

// Convert a GraphQL query "where: {}" argument to an "Expression" AST
fun graphqlWhereArgumentToExpression(whereArg: Argument): Expression {

    // Recursive function which walks the GraphQL Query Document AST
    // for the "where" argument, and constructs an equivalent relational expression AST
    fun go(objectValue: ObjectValue): Expression {
        val expr = objectValue.objectFields.map {
            when (it.name) {
                "_not" -> NOT(go(it.value as ObjectValue))

                "_and" -> (it.value as ArrayValue).values
                    .map { go(it as ObjectValue) }
                    .reduceRight(::AND) // Combine with logical AND

                "_or" -> (it.value as ArrayValue).values
                    .map { go(it as ObjectValue) }
                    .reduceRight(::OR) // Combine with logical OR

                else -> {
                    val name = it.name
                    val node = (it.value as ObjectValue).objectFields.first()
                    val value = graphqlValueToJava(node.value)
                    when (node.name) {
                        "_eq" -> EQ(COLUMN(name), LITERAL(value))
                        "_ne" -> NEQ(COLUMN(name), LITERAL(value))
                        "_lt" -> LT(COLUMN(name), LITERAL(value))
                        "_lte" -> LTE(COLUMN(name), LITERAL(value))
                        "_gt" -> GT(COLUMN(name), LITERAL(value))
                        "_gte" -> GTE(COLUMN(name), LITERAL(value))
                        "_in" -> IN(COLUMN(name), LITERAL(value))
                        "_nin" -> NIN(COLUMN(name), LITERAL(value))
                        "_is_null" -> IS_NULL(COLUMN(name))
                        else -> throw IllegalArgumentException("Unsupported operator: ${node.name}")
                    }
                }
            }
        }

        // Combine all top-level conditions with logical AND
        return expr.reduceRight(::AND)
    }

    return go(whereArg.value as ObjectValue)
}

In [6]:
val whereArg = getWhereArgumentFromGraphQLQuery(sampleQuery)
val testExpression = whereArg?.let { graphqlWhereArgumentToExpression(it) }

if (testExpression != null) {
    println(testExpression.prettyPrint())
}

AND(
    left=OR(
        left=EQ(
            left=COLUMN(
                name=DEPTNO
            ),
            right=LITERAL(
                value=20
            )
        ),
        right=EQ(
            left=COLUMN(
                name=DEPTNO
            ),
            right=LITERAL(
                value=30
            )
        )
    ),
    right=AND(
        left=GTE(
            left=COLUMN(
                name=SAL
            ),
            right=LITERAL(
                value=1500
            )
        ),
        right=OR(
            left=EQ(
                left=COLUMN(
                    name=JOB
                ),
                right=LITERAL(
                    value=SALESMAN
                )
            ),
            right=EQ(
                left=COLUMN(
                    name=JOB
                ),
                right=LITERAL(
                    value=MANAGER
                )
            )
        )
    )
)


# 3. Generate a Calcite "Row Expression" (RexNode) from our representation of the `WHERE` clause

Here we convert our "Expression" interface to the equivalent Calcite `RexNode`.

This is straightforward to do, because we assigned each of our interface types a corresponding `SqlOperator`.

We can just switch on the type of the expression and form the proper signature for `RelBuilder.call()`

In [7]:
import org.apache.calcite.tools.RelBuilder

fun Expression.toRexNode(builder: RelBuilder): RexNode = when (this) {
    is BinaryOperation -> {
        val left = left.toRexNode(builder)
        val right = right.toRexNode(builder)
        builder.call(operation, left, right)
    }
    is UnaryOperation -> {
        val operand = operand.toRexNode(builder)
        builder.call(operation, operand)
    }
    is COLUMN -> builder.field(name)
    is LITERAL -> builder.literal(value)
    else -> throw Exception("Impossible")
}

# 4. Create a Calcite "Relational Expression" (RelNode) from a combination of the `WHERE` "RexNode" and `SELECT` "RexNode"

Now, we need to perform the meat of the functionality.

In order to create a full Relational Expression in Calcite, we need an underlying datasource and schema for it to be written against. In the `org.apache.test` namespace there is a class called `CalciteAssert` that can wire up a sample database to Calcite for you.

It's important to know what `CalciteAssert` is doing here. What is going to happen behind the scenes is:
- Calcite will initialize a JDBC connection, called `CalciteConnection`
- A "root schema" of type `SchemaPlus` will be created. This is the value that will hold all datasources.
    - The "root schema" will become available under `CaliteConnection.getRootSchema()`
    - NOTE: You can manually create an empty "root schema" using `Frameworks.createRootSchema()`
- The test database + schema will be added to the "root schema"

Then we will use the `Frameworks` tool, to create a config object which initializes things like the Planner and many other necessary bits.

In [8]:
import org.apache.calcite.jdbc.CalciteConnection
import org.apache.calcite.sql.dialect.CalciteSqlDialect
import org.apache.calcite.sql.parser.SqlParser
import org.apache.calcite.test.CalciteAssert
import org.apache.calcite.tools.Frameworks
import org.apache.calcite.tools.RelBuilder
import org.apache.calcite.rel.rel2sql.RelToSqlConverter

// Start by using "calcite-testkit" CalciteAssert to create a test JDBC schema and Calcite root schema
val calciteAssert = CalciteAssert.that().with(CalciteAssert.SchemaSpec.JDBC_SCOTT)
val connection = calciteAssert.connect() as CalciteConnection

// Use the Framework utility, passing it the "connection.rootSchema" as the default schema
// Make sure to set SqlParser.withCaseSensitive(false) or else statements won't parse correctly
val config = Frameworks.newConfigBuilder()
    .defaultSchema(connection.rootSchema)
    .parserConfig(SqlParser.config().withCaseSensitive(false))
    .build()

// Create a Calcite "RelBuilder" using the FrameworkConfig
// This RelBuilder has the context of our JDBC schema in it
val relBuilder = RelBuilder.create(config)

// Convert GraphQL query to our "Expression" AST
val whereArg = requireNotNull(getWhereArgumentFromGraphQLQuery(sampleQuery))
val expression = graphqlWhereArgumentToExpression(whereArg)

// Scan the "JDBC_SCOTT.EMP" table
//
// - Filter by converting our "Expression" AST to
//   a "RexNode" via the "RelBuilder" we initialized
//
// - Select columns by iterating the GraphQL query selected
//   fields and converting to "RelBuilder.field()"
//
val relRoot = relBuilder.scan("JDBC_SCOTT", "EMP")
    .filter(expression.toRexNode(relBuilder))
    .project(
        getQuerySelectedFieldNames(sampleQuery).map {
            relBuilder.field(it)
        }
    )
    .build()

// Print out the query (logical) plan
println(relRoot.explain())

// Convert the RelNode of the query to equivalent SQL expression
val sqlNode = RelToSqlConverter(CalciteSqlDialect.DEFAULT).visitRoot(relRoot).asStatement()

// Print the equivalent SQL to check that our new frontend works as we expect
println("RelToSqlConverter result: ")
println(sqlNode.toSqlString(CalciteSqlDialect.DEFAULT))

LogicalProject(EMPNO=[$0], ENAME=[$1], JOB=[$2], MGR=[$3], SAL=[$5], COMM=[$6], DEPTNO=[$7])
  LogicalFilter(condition=[AND(SEARCH($7, Sarg[20, 30]), >=($5, 1500), SEARCH($2, Sarg['MANAGER':CHAR(8), 'SALESMAN']:CHAR(8)))])
    JdbcTableScan(table=[[JDBC_SCOTT, EMP]])

RelToSqlConverter result: 
SELECT "EMPNO", "ENAME", "JOB", "MGR", "SAL", "COMM", "DEPTNO"
FROM "SCOTT"."EMP"
WHERE "DEPTNO" IN (20, 30) AND "SAL" >= 1500 AND "JOB" IN ('MANAGER', 'SALESMAN')


# 5. Execute the RelNode against the data source we are trying to query against

In [9]:
import org.apache.calcite.tools.RelRunner

// We use the "RelRunner" class to execute RelNode expressions
val runner: RelRunner = connection.unwrap(RelRunner::class.java)

// Execute our query RelNode and print the results
runner.prepareStatement(relRoot).executeQuery().let { 
    val md = it.metaData
    val columns = md.columnCount
    while (it.next()) {
        for (i in 1..columns) {
            if (i > 1) print(",  ")
            System.out.print(md.getColumnName(i) + " " + it.getString(i))
        }
        println("")
    }
 }

EMPNO 7499,  ENAME ALLEN,  JOB SALESMAN,  MGR 7698,  SAL 1600.00,  COMM 300.00,  DEPTNO 30
EMPNO 7566,  ENAME JONES,  JOB MANAGER,  MGR 7839,  SAL 2975.00,  COMM null,  DEPTNO 20
EMPNO 7698,  ENAME BLAKE,  JOB MANAGER,  MGR 7839,  SAL 2850.00,  COMM null,  DEPTNO 30
EMPNO 7844,  ENAME TURNER,  JOB SALESMAN,  MGR 7698,  SAL 1500.00,  COMM 0.00,  DEPTNO 30
